In [214]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer, sent_tokenize , TreebankWordTokenizer,WordPunctTokenizer
from nltk.corpus import stopwords
import string
import contractions
from spellchecker import SpellChecker
import re
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [215]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC2\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [216]:
spell = SpellChecker()

In [217]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [218]:


def text_preprocessing(text,lang,tokenizer=WhitespaceTokenizer(),stopw=True,punctuation=True,lowercase=True,lemmatize=True,spelling=True,expand_contraction=True,urls=True):
    if lang.lower()=='english':
        stopword =stopwords.words('english')
        lemmatizer = WordNetLemmatizer()
        spell = SpellChecker()
    else :#if lang="french"
        stopword = stopwords.words('french')
        lemmatizer = FrenchLefffLemmatizer()
        spell = SpellChecker(language="fr")
    if lowercase==True:
        #lowercase the text 
        text = text.lower()
    if urls==True:
        #remove urls
        text=remove_urls(text)
    #tokenize the text 
    tokens = tokenizer.tokenize(text)
    if expand_contraction==True:
        #expand contractions
        tokens = [contractions.fix(token) for token in tokens]
    if punctuation==True:
        #remove punctuation
        tokens = [token for token in tokens if token not in string.punctuation]
    if stopwords==True:
        #remove stopwords
        tokens = [token for token in tokens if token not in stopword]
    if spelling==True:
        #spell check:
        tokens = [spell.correction(token) for token in tokens]
    if lemmatize==True:
        #lemmatization : 
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(word for word in tokens)

    



In [219]:
file = 'C:/Users/PC2/Downloads/test-ex.txt'
f=open(file,'r')
data = f.read()
data
#re.sub(r'\r\n\s*\r\n', '\r\n', data)

'\n\nGisevciconia\n\nAERO SOLUTIONS\n\nObjet : ordre de virement\n\n \n\nMessieurs ;\n\n \nPar le dÃ©bit de notre compte :_RIB: 022 222 222 172 22 29576475 53\n\n \n\nswt\nAGILISYS AERO SOLUTIONS\nSWIFT: AGILAMCXXX\n| De virer le Montant de : 89 250,00 Dirhams (Quatre-vingt-neuf Mille deux cent cinquante Dirhams ).\n| Au profit de i\nCARLEX Export Switzerland\nIBAN : CH35 0483 555 555 0266600 0\n\nSWIFT : CRESCHZZ80A\n\n \n\nMotif : 22A005\n\nSignature :\n\n \n\nAGILISYS INDUSTRIES SARL | SiÃ©ge social ; Angle nÂ°2 rue ibn aicha et bd Abdelkrim el khattabi imm le Paris 2Â°â„¢* Ã©tage appt nÂ°23 Gueliz â€”Marrakech\nT: +212 (0) 999 99 6161 l F : +212 (0) 999 999 130 !\n\nSARL au Capital de 500.000 000 dhs | RCNÂ°35 33 | IF nÂ° 3939333 | T.PnÂ° 3939 | CNSS:939\nMT-103-STP 2838383393 (MT Print)\n\npoet trot ccc cs econ Instance Type and Transmission --\nOriginal received from \n\n '

In [220]:
text_preprocessing(data,'fre')

'gisevciconia zero solution objet ordre de virement Monsieur par le débit de notre compte :_rib: 022 222 222 172 22 29576475 53 set agilisys zero solution swift agilamcxxx de virer le montant de 89 250,00 dirais (quatre-vingt-neuf mille deux cent cinquante dirais de au profit de i parler export switzerland ian chez 0483 555 555 0266600 0 swift creschzz80a motif 22a005 signature agilisys industrie sara siège social angle nâ°2 rue ibn aisha et boulevard abdelkrim el khattabi millimètre le pari 2â°â„¢* étage appât nâ°23 gueulez â€”marrakech tu +212 (0) 999 99 6161 l f +212 (0) 999 999 130 sara au capital de 500.000 000 des rcnâ°35 33 if ne 3939333 t.pnâ° 3939 cnss:939 mt-103-stp 2838383393 et point pote trot cc cs con instance type and transmission de original receive from'

In [221]:
WhitespaceTokenizer().tokenize(data)

['Gisevciconia',
 'AERO',
 'SOLUTIONS',
 'Objet',
 ':',
 'ordre',
 'de',
 'virement',
 'Messieurs',
 ';',
 'Par',
 'le',
 'dÃ©bit',
 'de',
 'notre',
 'compte',
 ':_RIB:',
 '022',
 '222',
 '222',
 '172',
 '22',
 '29576475',
 '53',
 'swt',
 'AGILISYS',
 'AERO',
 'SOLUTIONS',
 'SWIFT:',
 'AGILAMCXXX',
 '|',
 'De',
 'virer',
 'le',
 'Montant',
 'de',
 ':',
 '89',
 '250,00',
 'Dirhams',
 '(Quatre-vingt-neuf',
 'Mille',
 'deux',
 'cent',
 'cinquante',
 'Dirhams',
 ').',
 '|',
 'Au',
 'profit',
 'de',
 'i',
 'CARLEX',
 'Export',
 'Switzerland',
 'IBAN',
 ':',
 'CH35',
 '0483',
 '555',
 '555',
 '0266600',
 '0',
 'SWIFT',
 ':',
 'CRESCHZZ80A',
 'Motif',
 ':',
 '22A005',
 'Signature',
 ':',
 'AGILISYS',
 'INDUSTRIES',
 'SARL',
 '|',
 'SiÃ©ge',
 'social',
 ';',
 'Angle',
 'nÂ°2',
 'rue',
 'ibn',
 'aicha',
 'et',
 'bd',
 'Abdelkrim',
 'el',
 'khattabi',
 'imm',
 'le',
 'Paris',
 '2Â°â„¢*',
 'Ã©tage',
 'appt',
 'nÂ°23',
 'Gueliz',
 'â€”Marrakech',
 'T:',
 '+212',
 '(0)',
 '999',
 '99',
 '6161',
 'l

In [222]:
from spacy.lang.fr import French
from spacy.lang.en import English
nlp_en = spacy.load("en_core_web_sm", disable=['parser', 'tagger', 'ner'])
nlp_fr = spacy.load("fr_core_news_sm", disable=['parser', 'tagger', 'ner'])
def spacy_preprocessing(text,lang,lowercase=True,stopw=True,punctuation=True,alphabetic=True,lemmatize=True,):
    if lang =="en":
        nlp = nlp_en
    else :
        nlp = nlp_fr
    if lowercase:
        text = text.lower()
    #tokenize with spacy's default tokenizer
    tokens = nlp(text)
    if stopw :
        tokens = [token for token in tokens if not token.is_stop]
    if lemmatize :
        tokens = [token.lemma_.strip() for token in tokens]
    if punctuation :
        tokens = [re.sub('<[^>]*>', '', token) for token in tokens]
    if alphabetic:
        tokens = [re.sub('[\W]+','',token.lower()) for token in tokens]
    return ' '.join(word for word in tokens)
    
    

In [227]:

spacy_preprocessing(data,'fr')

' gisevciconia  aero solutions  objet  ordre virement  monsieur   dã  bit compte  _ rib  022 222 222 172 22 29576475 53  swt  agilisy aero solution  swift  agilamcxx   virer montant  89 25000 dirham  quatrevingtneuf dirham     profit  carlex export switzerland  iban  ch35 0483 555 555 0266600 0  swift  creschzz80a  motif  22a005  signature   agilisys industrie sarl  siã  ge social  angle nâ degré 2 rue ibn aicha bd abdelkrim el khattabi imm pari 2â degré â  ã  tage appt nâ degré 23 gueliz âmarrakech  t  212  0  999 99 6161 l f  212  0  999 999 130   sarl capital 500000 000 dhs  rcnâ degré 35 33  if nâ degré 3939333  tpnâ degré 3939  cnss939  mt103stp 2838383393  mt print   poet trot ccc cs econ instance type and transmission   original received from '